In [2]:
import pandas as pd
import re

In [3]:
urls1 = pd.read_csv("../data/posco_url_1001_1014.csv")
urls2 = pd.read_csv("../data/posco_url_1014_1025.csv")
urls3 = pd.read_csv("../data/posco_url_1026_1031.csv")

In [7]:
urls = pd.concat([urls1, urls2, urls3]).drop_duplicates()

In [8]:
# 댓글을 달 빈 리스트를 생성합니다.
body_lst = []
reply_count_lst = []
uid_lst = []
like_lst = []
dislike_lst = []
c_time_lst= []
cid_lst = []
pid_lst = []



# 라이브러리를 로드합니다.
from bs4 import BeautifulSoup
import requests
import re
import sys
import pprint
from tqdm import tqdm

total_df = pd.DataFrame()
 
for url, title, time, media in tqdm(zip(urls['0'], urls['1'], urls['2'], urls['3'])):

    body_lst = []
    reply_count_lst = []
    uid_lst = []
    like_lst = []
    dislike_lst = []
    c_time_lst= []
    cid_lst = []
    pid_lst = []

    try:
        oid = re.findall('article/([^\*]*)/', str(url))[0]
        aid = re.findall(f'{oid}/([^\*]*)[?]', str(url))[0]
    except:
        print(url)
        continue

    sid = url[-3:]
    page = 1
    header = {
        "User-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36",
        "referer": url,
    }
    
    while True:
        c_url = "https://apis.naver.com/commentBox/cbox/web_neo_list_jsonp.json?ticket=news&templateId=default_society&pool=cbox5&_callback=jQuery1707138182064460843_1523512042464&lang=ko&country=&objectId=news" + oid + "%2C" + aid + "&categoryId=&pageSize=20&indexSize=10&groupId=&listType=OBJECT&pageType=more&page=" + str(
            page) + "&refresh=false&sort=FAVORITE"
        # 파싱하는 단계입니다.
        r = requests.get(c_url, headers=header)
        cont = BeautifulSoup(r.content, "html.parser")
        total_comm = str(cont).split('comment":')[1].split(",")[0]
    
        body = re.findall('"contents":([^\*]*),"userIdNo"', str(cont))
        reply_count = re.findall('"replyCount":([^\*]*),"replyAllCount"', str(cont))
        uid = re.findall('"userIdNo":([^\*]*),"exposedUserIp"', str(cont))
        like = re.findall('sympathyCount":([^\*]*),"antipathyCount"', str(cont))
        dislike = re.findall('"antipathyCount":([^\*]*),"hideReplyButton"', str(cont))
        c_time = re.findall('"modTime":([^\*]*),"modTimeGmt"', str(cont))
        cid = re.findall('commentNo":"([^\*]*)","parentCommentNo"', str(cont))
        pid = re.findall('"parentCommentNo":"([^\*]*)","replyLevel"', str(cont))

        comment_num = len(body)

        # 댓글을 리스트에 중첩합니다.
        body_lst.append(body)
        reply_count_lst.append(reply_count)
        uid_lst.append(uid)
        like_lst.append(like)
        dislike_lst.append(dislike)
        cid_lst.append(cid)
        pid_lst.append(pid)
        c_time_lst.append(c_time)


        # 한번에 댓글이 20개씩 보이기 때문에 한 페이지씩 몽땅 댓글을 긁어 옵니다.
        if int(total_comm) <= ((page) * 20):
            break
        else:
            page += 1
    
    
    # 여러 리스트들을 하나로 묶어 주는 함수입니다.
    def flatten(l):
        flatList = []
        for elem in l:
            # if an element of a list is a list
            # iterate over this list and add elements to flatList
            if type(elem) == list:
                for e in elem:
                    flatList.append(e)
            else:
                flatList.append(elem)
        return flatList
    
     
    # 리스트 결과입니다.
    
    allCommetns = flatten(body_lst)
    allreply_count = flatten(reply_count_lst)
    alluid = flatten(uid_lst)
    alllike = flatten(like_lst)
    alldislike = flatten(dislike_lst)
    allctime = flatten(c_time_lst)
    allcid = flatten(cid_lst)
    allpid = flatten(pid_lst)

    url_df = pd.DataFrame([allCommetns, allreply_count,alluid, alllike, alldislike, allctime, allcid, allpid]).T
    url_df['sid'] = sid
    url_df['time'] = time
    url_df['title'] = title
    url_df['media'] = media

    url_df.columns = ['comment', 'reply', 'author', 'like', 'dislike', 'c_time', 'cid', 'pid', 'sid', 'time', 'title', 'media']

    total_df = pd.concat([total_df, url_df])

#print(allCommetns)
# print(len(allCommetns))
# print(len(allreply_count))
# print(len(alluid))
# print(len(alllike))
# print(len(alldislike))
# print(len(allctime))
# print(len(allcid))
# print(len(allpid))

1843it [07:09,  4.29it/s]


In [9]:
total_df

,comment,reply,author,like,dislike,c_time,cid,pid,sid,time,title,media
0,"""전쟁폐허에서 오직 잘사는나라 건설 일념으로 오늘날 선진국에 오른 바탕을 다져놓은 ...",0,"""eCiha""",25,6,"""2022-10-13T15:49:35+0900""",771976322070085934,771976322070085934,101,2022.10.13. 오후 2:56,"러·우 전쟁에… ‘그린스틸’ 투자한 포스코, 몸값 치솟는다",조선일보
1,"""포스코, 박정희 대통령👍""",0,"""3HYWb""",5,2,"""2022-10-13T15:57:21+0900""",771976822316335281,771976822316335281,101,2022.10.13. 오후 2:56,"러·우 전쟁에… ‘그린스틸’ 투자한 포스코, 몸값 치솟는다",조선일보
2,"""철강사 세계 경쟁력 12년간 연속 1위하고 있는 기업이다. 그냥 두면 너무나 잘하...",0,"""nAuD""",2,1,"""2022-10-13T19:11:15+0900""",771989313775730827,771989313775730827,101,2022.10.13. 오후 2:56,"러·우 전쟁에… ‘그린스틸’ 투자한 포스코, 몸값 치솟는다",조선일보
3,"""포스코 화이팅이다.""",0,"""36jfp""",1,0,"""2022-10-13T16:09:43+0900""",771977618495898044,771977618495898044,101,2022.10.13. 오후 2:56,"러·우 전쟁에… ‘그린스틸’ 투자한 포스코, 몸값 치솟는다",조선일보
4,"""꼭 세계 시장의 표준을 선점하길...""",0,"""25dMA""",1,0,"""2022-10-13T15:38:47+0900""",771975626016948620,771975626016948620,101,2022.10.13. 오후 2:56,"러·우 전쟁에… ‘그린스틸’ 투자한 포스코, 몸값 치솟는다",조선일보
...,...,...,...,...,...,...,...,...,...,...,...,...
1,"""돈맥경화 확산' 10대그룹도 유동성 점검 강화…\""투자 줄이고 현금 확보\""......",0,"""1QhiX""",0,0,"""2022-10-26T22:10:22+0900""",773206880632701202,773206880632701202,101,2022.10.26. 오후 3:19,"'돈맥경화 확산' 10대그룹도 유동성 점검 강화…""투자 줄이고 현금 확보""",뉴스1
2,"""중견기업 이미 도산 임박했고 대기업군 도차도 흔들흔들. 시장에서 신뢰를 잃으면 이...",0,"""EbAB""",0,0,"""2022-10-26T16:59:32+0900""",773186855297351993,773186855297351993,101,2022.10.26. 오후 3:19,"'돈맥경화 확산' 10대그룹도 유동성 점검 강화…""투자 줄이고 현금 확보""",뉴스1
0,"""승강기 자동호출은 좋은데 거주층 자동연결은 빼는게좋다? 센서감지로 인한 빈승강기...",0,"""3KQPW""",1,0,"""2022-10-26T09:31:54+0900""",773158016320012748,773158016320012748,101,2022.10.26. 오전 9:20,'둔산 더샵 엘리프' 편안하게 내집까지... 스마트홈으로 주목,아시아경제
0,"""헛소리""",0,"""bKZ3t""",0,0,"""2022-10-27T21:00:48+0900""",773295170077589915,773295170077589915,101,2022.10.26. 오후 2:30,[월간중앙] 특별 인터뷰 | ‘세계 금융위기’ 대처한 전광우 前 금융위원장의 죽비 소리,중앙일보


In [10]:
#print(total_df.groupby('title').size().sort_values(ascending= False))
print(total_df.groupby('media').size().sort_values(ascending= False))

media
동아일보           1088
한국경제            827
오마이뉴스           802
조선일보            635
조선비즈            630
중앙일보            609
매일경제            581
한국일보            539
연합뉴스            382
서울경제            320
뉴스1             262
뉴시스             235
디지털타임스          225
노컷뉴스            186
부산일보            186
파이낸셜뉴스          184
이데일리            172
아시아경제           152
머니투데이           139
매경이코노미          104
MBC             102
문화일보            101
비즈니스워치          100
매일신문             96
헤럴드경제            95
데일리안             80
서울신문             74
한경비즈니스           67
경향신문             59
프레시안             58
더팩트              57
이코노미스트           54
시사저널             53
SBS Biz          50
TV조선             49
채널A              49
아이뉴스24           47
ZDNet Korea      43
국민일보             42
블로터              36
KBS              33
디지털데일리           28
세계일보             26
한국경제TV           25
한겨레              25
YTN              18
전자신문             18
조세일보             18
kbc광주방송          14
머니S           

In [11]:
total_df.reset_index(drop = True, inplace = True)

In [10]:
total_df.to_pickle("../result/naver_POSCO_comments_22_10.pkl")
total_df.to_csv("../result/naver_POSCO_comments_22_10.csv")

In [12]:
total_df

,comment,reply,author,like,dislike,c_time,cid,pid,sid,time,title,media
0,"""최정우 회장은 저거 말고도 물적분할건과 주주들에게 약속한 배당도 안주고 또 회사 ...",0,"""4tsNy""",38,3,"""2022-09-30T11:21:14+0900""",770753006533607540,770753006533607540,102,2022.09.30. 오전 10:09,"[단독]최정우 향한 '포스코 자사주 매입' 의혹, 중앙지검 반부패3부에서 재수사 착수",시사저널
1,"""강백신 부장검사를 배치하다니.. 엄청나다 엄청나..\n 친윤 변호사들 몇 명이라도...",0,"""3rVbk""",27,2,"""2022-09-30T11:54:47+0900""",770755168059785600,770755168059785600,102,2022.09.30. 오전 10:09,"[단독]최정우 향한 '포스코 자사주 매입' 의혹, 중앙지검 반부패3부에서 재수사 착수",시사저널
2,"""포스코창립이래 최악 무능 최정우 부산대 문과출신의 몰락. 포스코 젊은인재들 떠나...",0,"""6wZ6I""",12,1,"""2022-09-30T23:08:05+0900""",770798544830332975,770798544830332975,102,2022.09.30. 오전 10:09,"[단독]최정우 향한 '포스코 자사주 매입' 의혹, 중앙지검 반부패3부에서 재수사 착수",시사저널
3,"""최정우로인해 포스코주주들은 피눈물흘리는중""",0,"""1Ucu4""",12,3,"""2022-09-30T12:00:02+0900""",770755506774999057,770755506774999057,102,2022.09.30. 오전 10:09,"[단독]최정우 향한 '포스코 자사주 매입' 의혹, 중앙지검 반부패3부에서 재수사 착수",시사저널
4,"""제발 처벌좀...""",0,"""1pfiN""",10,2,"""2022-09-30T11:38:00+0900""",770754086499778571,770754086499778571,102,2022.09.30. 오전 10:09,"[단독]최정우 향한 '포스코 자사주 매입' 의혹, 중앙지검 반부패3부에서 재수사 착수",시사저널
...,...,...,...,...,...,...,...,...,...,...,...,...
27098,"""국가 전략 기동 부대 답네""",0,"""1sBeJ""",1,0,"""2022-09-07T00:15:33+0900""",768576380492317054,768576380492317054,102,2022.09.06. 오후 3:25,"해병대 1사단, 상륙돌격장갑차 투입...물에 갇힌 포항 주민들 구조 '맹활약'",프레시안
27099,"""멋졌다. 저런거에 세금 쓴다는 빙구는 세금이나 제대로 내고 말혀라.""",1,"""3ldIZ""",1,0,"""2022-09-06T17:56:50+0900""",768551981823297713,768551981823297713,102,2022.09.06. 오후 3:25,"해병대 1사단, 상륙돌격장갑차 투입...물에 갇힌 포항 주민들 구조 '맹활약'",프레시안
27100,"""법무부는 술판벌였다며?ㅋㅋㅋ""",0,"""2BKjb""",0,1,"""2022-09-06T20:28:36+0900""",768561759266013801,768561759266013801,102,2022.09.06. 오후 3:25,"해병대 1사단, 상륙돌격장갑차 투입...물에 갇힌 포항 주민들 구조 '맹활약'",프레시안
27101,"""세금을 물쓰듯이쓰는구나...또물가올리겠네...""",0,"""26OV0""",0,8,"""2022-09-06T17:54:08+0900""",768551808061669772,768551808061669772,102,2022.09.06. 오후 3:25,"해병대 1사단, 상륙돌격장갑차 투입...물에 갇힌 포항 주민들 구조 '맹활약'",프레시안
